# Convolutional Neural Network

This [tutorial](https://www.superdatascience.com/blogs/the-ultimate-guide-to-convolutional-neural-networks-cnn) contains an intuitive explanation of CNNs

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Dataset is too large for uploading onto Colab. This notebook will be run with the dataset located locally.

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

This is done to prevent overfitting of the model. Image transformations are going to be performed onto the training data. Image augmentation is the term that is commonly used. The [keras library](https://keras.io/api/preprocessing/image/) is great for exploring image pre-processing modules

In [ ]:
'''
rescale is a feature scaller that scales each pixel by dividing it by 255
shear_range, zoom_range and horizontal_flip are image translators
'''

#Creting an instance of the ImageDataGenerator class

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#Applying the flow_from_directory method on the train_datagen object

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

### Preprocessing the Test set

In [ ]:
'''
images in the test set are only scaled. We don't want to transform them
'''

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
'''
filter - number of feature detectors
kernel_size - size of one row of the feature detector
activation - we want to keep activation as a rectified linear unit during the hidden layers
'''
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) 

 ### Step 2 - Pooling

In [ ]:
'''
A relatively small pooling parameter is used for finer image detail
'''
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
'''
input_shape parameter is removed because convolution layer is not an input.
Another pooling layer is also added to improve robustness
'''

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
'''
We add a dense layer of 128 neurons 
'''

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image

#Loading test image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))

#Transform image to array
test_image = image.img_to_array(test_image)

#Expanding the dimensions of the array
test_image = np.expand_dims(test_image, axis = 0)

#Obtaining the results
result = cnn.predict(test_image)

#Check which set of images belong to which classification
training_set.class_indices

'''
result has 2D representation. The first is the batch and the second is the prediction of the image
'''

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)